In [ ]:
#importing required modules

In [1]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np
os.chdir("C:/Users/karthi/Desktop/image_classifier")
ses=tf.Session()

In [ ]:
#creating required tensorflow variables ie.weights and biases

In [2]:
x=tf.placeholder("float32",[1,128,128,3])
y=tf.placeholder('float32',[1,2])
conv_layer1={"weights":tf.Variable(tf.truncated_normal([3,3,3,3])),"bias":tf.Variable(tf.random_normal([3]))}
conv_layer2={"weights":tf.Variable(tf.truncated_normal([5,5,3,32])),"bias":tf.Variable(tf.random_normal([32]))}
conv_layer3={"weights":tf.Variable(tf.truncated_normal([6,6,32,64])),"bias":tf.Variable(tf.random_normal([64]))}
fc_layer1={"weights":tf.Variable(tf.truncated_normal([32*32*64,600])),"bias":tf.Variable(tf.random_normal([600]))}
out_layer={"weights":tf.Variable(tf.truncated_normal([600,2])),"bias":tf.Variable(tf.random_normal([2]))}

In [ ]:
#loading the tensorflow model (which is already pre_trained using image_classifier.pynb program)

In [7]:
ses.run(tf.global_variables_initializer())
saver=tf.train.Saver()
saver.restore(ses,"model/trained_model.ckpt")

INFO:tensorflow:Restoring parameters from model/trained_model.ckpt


In [ ]:
#defining the convolution network

In [8]:
def conv_classify(x):
    print("==========================================================")
    con1=tf.nn.conv2d(x,conv_layer1["weights"],strides=[1,1,1,1],padding="SAME")+conv_layer1["bias"]
    print("convolution layer1 {}   filter  3X3".format(con1.shape))
    con1=tf.nn.relu(con1)
    print("relu {}".format(con1.shape))
    print("==========================================================")
    con2=tf.nn.conv2d(con1,conv_layer2["weights"],strides=[1,1,1,1],padding="SAME")+conv_layer2["bias"]
    print("convolution layer2 {}   filter  5X5".format(con2.shape))
    con2=tf.nn.relu(con2)
    print("relu {}".format(con2.shape))
    con2=tf.nn.max_pool(con2,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
    print("maxpool layer {}   filter  2X2".format(con2.shape))
    print("==========================================================")
    con3=tf.nn.conv2d(con2,conv_layer3["weights"],strides=[1,1,1,1],padding="SAME")+conv_layer3["bias"]
    print("convolution layer3 {}   filter  6X6".format(con3.shape))
    con3=tf.nn.relu(con3)
    print("relu {}".format(con3.shape))
    con3=tf.nn.max_pool(con3,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
    print("maxpool layer {}   filter  2X2".format(con3.shape))
    print("==========================================================")
    con3=tf.reshape(con3,[-1,32*32*64])
    fc=tf.add(tf.matmul(con3,fc_layer1["weights"]),fc_layer1["bias"])
    fc=tf.nn.sigmoid(fc)
    print("fully connected layer {}".format(fc.shape))
    fc=tf.nn.dropout(fc,0.75)
    out=tf.add(tf.matmul(fc,out_layer["weights"]),out_layer["bias"])
    out=tf.nn.sigmoid(out)
    print("output layer {}".format(out.shape))
    print("==========================================================")
    return out

In [ ]:
#classification

In [9]:
pred=conv_classify(x)
files=os.listdir("test_images/")
for each in files:
    image=Image.open("test_images/{}".format(each))
    x_data=np.array(image.getdata()).reshape([1,128,128,3]).astype("float32")
    val=ses.run(pred,feed_dict={x:x_data})
    print(val)
    maxi=max(val[0])
    if val[0][0]==maxi:
        print("{} is cat".format(each)) 
    else:
        print("{} is car".format(each))

convolution layer1 (1, 128, 128, 3)   filter  3X3
relu (1, 128, 128, 3)
convolution layer2 (1, 128, 128, 32)   filter  5X5
relu (1, 128, 128, 32)
maxpool layer (1, 64, 64, 32)   filter  2X2
convolution layer3 (1, 64, 64, 64)   filter  6X6
relu (1, 64, 64, 64)
maxpool layer (1, 32, 32, 64)   filter  2X2
fully connected layer (1, 600)
output layer (1, 2)
[[  1.30740176e-11   1.00000000e+00]]
1.png is car
[[  1.00000000e+00   2.14821650e-13]]
2.png is cat
[[  1.00000000e+00   1.53626303e-07]]
gvhvkvkj.png is cat
[[  1.00000000e+00   8.92891758e-18]]
obj14__175.png is cat
[[  9.99991298e-01   2.15046995e-04]]
obj14__245.png is cat
[[  9.89270329e-01   8.49722255e-07]]
obj14__70.png is cat
[[  3.07103204e-10   9.99999404e-01]]
obj6__230.png is car
[[  1.18300022e-07   9.99998331e-01]]
obj6__260.png is car
[[  1.45234020e-12   1.00000000e+00]]
obj6__355.png is car
[[  3.82087746e-05   1.00000000e+00]]
z.png is car
